In [ ]:
from navground import sim
import h5py
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial.distance as dist

import iblofunmatch.inter as ibfm
output_dir="output"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
side_length = 10
yaml = f"""
steps: 10000
time_step: 0.1
record_safety_violation: true
record_task_events: true
record_pose: true
runs: 1
save_directory: .
scenario:
  type: Cross
  side: {side_length}
  agent_margin: 0.2
  add_safety_to_agent_margin: true
  tolerance: 0.6
  position_noise: 0.2
  groups:
    - number: 30
      type: thymio
      control_period: 0.1
      behavior:
        type: HL
        safety_margin: 0.1 
      radius: 0.2
      kinematics:
        type: 2WDiff
        max_speed: 1.0
        wheel_axis: 2
      state_estimation:
        type: Bounded
        range: 2.0
"""

In [ ]:
experiment = sim.load_experiment(yaml)
experiment.run()
data = h5py.File(experiment.path)
print(data.keys())
run = data["run_0"]
ps = run['poses']

Consider two close timesteps in the experiment. Was the movement ordered or not?

In [ ]:
start_time = 100
end_time = 125 # long wait
X = ps[start_time][:,[0,1]]
Y = ps[end_time][:,[0,1]]
fig, ax = plt.subplots(ncols=3, figsize=(18,6))
colorlist = ["red", "black", "blue"]
# Scatter points 
ax[0].scatter(X[:,0], X[:,1], c=colorlist[0])
ax[2].scatter(Y[:,0], Y[:,1], c=colorlist[2])
# plot both point clouds in the middle
ax[1].scatter(X[:,0], X[:,1], c=colorlist[0], s=30, zorder=3, marker="x")
ax[1].scatter(Y[:,0], Y[:,1], c=colorlist[2], s=30, zorder=3, marker="+")
for start, end in zip(X, Y):
    ax[1].plot([start[0], end[0]], [start[1], end[1]], c="black")
plt.savefig("points_matching_large_timestep.png")

## Compute Block Function using minimum distances in middle

In [ ]:
Dist_X = dist.squareform(dist.pdist(X))
Dist_Y = dist.squareform(dist.pdist(Y))
Dist_Z = np.minimum(Dist_X, Dist_Y)
subset_indices = list(range(Dist_X.shape[0]))
ibfm_bfun = [
    ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, subset_indices, output_dir),
    ibfm.get_IBloFunMatch_subset(Dist_Y, Dist_Z, subset_indices, output_dir)
]

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(10, 2))
ibfm.plot_matching(ibfm_bfun[0], ax[[0,1]], fig, dim=0)
ibfm.plot_matching(ibfm_bfun[1], ax[[2,3]], fig, dim=0)
plt.savefig("match_0_large.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ibfm.plot_XYZ_matching_0(ibfm_bfun, ax)
max_x = max(np.max(ibfm_bfun[0]["S_barcode_0"]), np.max(ibfm_bfun[1]["X_barcode_0"]))
ax.set_xlim([-max_x*1.1,max_x*1.1])
plt.savefig("diagrams_0_match_large_timestep.png")

In [ ]:
Print barcode change across a small timestep.

In [ ]:
start_time = 100
end_time = 105 # long wait
X = ps[start_time][:,[0,1]]
Y = ps[end_time][:,[0,1]]
fig, ax = plt.subplots(ncols=3, figsize=(18,6))
colorlist = ["red", "black", "blue"]
# Scatter points 
ax[0].scatter(X[:,0], X[:,1], c=colorlist[0])
ax[2].scatter(Y[:,0], Y[:,1], c=colorlist[2])
# plot both point clouds in the middle
ax[1].scatter(X[:,0], X[:,1], c=colorlist[0], s=30, zorder=3, marker="x")
ax[1].scatter(Y[:,0], Y[:,1], c=colorlist[2], s=30, zorder=3, marker="+")
for start, end in zip(X, Y):
    ax[1].plot([start[0], end[0]], [start[1], end[1]], c="black")
plt.savefig("points_matching_small_timestep.png")

Dist_X = dist.squareform(dist.pdist(X))
Dist_Y = dist.squareform(dist.pdist(Y))
Dist_Z = np.minimum(Dist_X, Dist_Y)
subset_indices = list(range(Dist_X.shape[0]))
ibfm_bfun = [
    ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, subset_indices, output_dir),
    ibfm.get_IBloFunMatch_subset(Dist_Y, Dist_Z, subset_indices, output_dir)
]

fig, ax = plt.subplots(figsize=(10,10))
ibfm.plot_XYZ_matching_0(ibfm_bfun, ax)
max_x = max(np.max(ibfm_bfun[0]["S_barcode_0"]), np.max(ibfm_bfun[1]["X_barcode_0"]))
ax.set_xlim([-max_x*1.1,max_x*1.1])
plt.savefig("diagrams_0_match_small_timestep.png")